# Extracción y descripción de los datos

Empiezo importando las librerías que voy a utilizar.

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from pydub import AudioSegment
from pydub.utils import make_chunks

import re

Continuo creando 3 funciones con las que voy a hacer los siguiente:
 - La primera divide un archivo audio de larga duración en pequeños audios de 10 segundos de duración, y los guarda en la carpeta que le he especificado.
 - La segunda accede a cada uno de esos audios de 10 segundos y saca las features características de dicho archivo, y en base a su nombre le asigna una categoría y otra.
 - La tercera simplemente me enseña las principales características y métricas de la base de datos.

Finalmente, guardo el el dataset en mi carpeta como un CSV.

Dado que la base de datos ya la tengo creada, este JN lo voy a aplicar sobre audios que luego voy a utilizar para predecir con mi modelo y ver si es lo suficientemente bueno.

In [3]:
def decompose_files(data_dir, audio_files):
    for j in range(len(audio_files)):
        myaudio = AudioSegment.from_file(data_dir + '{}'.format(os.listdir(data_dir)[j])) 
        chunk_length_ms = 10000
        chunks = make_chunks(myaudio, chunk_length_ms)

        for i, chunk in enumerate(chunks):
            chunk_name = "{}{}.wav".format(os.listdir(data_dir)[j], i)
            print("exporting", chunk_name)
            chunk.export('./sounds2/{}'.format(chunk_name), format="mp3")
            
decompose_files('./sounds/', glob('./sounds' + '/*'))

exporting washing_machine_4-1.wav0.wav
exporting washing_machine_4-1.wav1.wav
exporting washing_machine_4-1.wav2.wav
exporting washing_machine_4-1.wav3.wav
exporting washing_machine_4-1.wav4.wav
exporting washing_machine_4-1.wav5.wav
exporting washing_machine_4-1.wav6.wav
exporting washing_machine_4-1.wav7.wav
exporting washing_machine_4-1.wav8.wav
exporting washing_machine_4-1.wav9.wav
exporting washing_machine_4-1.wav10.wav
exporting washing_machine_4-1.wav11.wav
exporting washing_machine_4-1.wav12.wav
exporting washing_machine_4-1.wav13.wav
exporting washing_machine_4-1.wav14.wav
exporting washing_machine_4-1.wav15.wav
exporting washing_machine_4-1.wav16.wav
exporting washing_machine_4-1.wav17.wav
exporting washing_machine_4-1.wav18.wav
exporting washing_machine_4-1.wav19.wav
exporting washing_machine_4-1.wav20.wav
exporting washing_machine_4-1.wav21.wav
exporting washing_machine_4-1.wav22.wav
exporting washing_machine_4-1.wav23.wav
exporting washing_machine_4-1.wav24.wav
exporting 

In [10]:
def get_features(data_dir, audio_files):
    momentos = {'ducha': 'Ducha', 'cena': 'Cena', 'washing': 'Lavadora', 'vacuum': 'Aspiradora', 
                'shaver': 'Afeitadora', 'hairdryer': 'Secador_pelo', 'airconditioner': 'Aire_acondicionado', 
                'cellphone': 'Telefono', 'comp': 'Tecleo', 'silence': 'Silencio', 'dryer': 'Secadora', 
                'blender': 'Licuadora', 'doorbell': 'Timbre', 'alarm': 'Alarma', 'faucet': 'Grifo', 
                'microwave': 'Microondas'}
    features = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i], )
        name = os.listdir(data_dir)[i]
        pattern = "[._][\w]+"
        name = re.sub(pattern, '', name)
        if name in momentos.keys():
            momento = momentos.get(name)
        else:
            momento = 'Otro'
        D = np.abs(librosa.stft(y))
        mfcc = np.ndarray.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20))
        features.append([D, mfcc, momento])
    return pd.DataFrame(features, columns=['D', 'mfcc', 'momento'])

data = get_features('./sounds2/', glob('./sounds2' + '/*.wav'))

In [109]:
def get_features(data_dir, audio_files):
    momentos = {'ducha': 'Ducha', 'cena': 'Cena', 'washing': 'Lavadora', 'vacuum': 'Aspiradora', 
                'shaver': 'Afeitadora', 'hairdryer': 'Secador_pelo', 'airconditioner': 'Aire_acondicionado', 
                'cellphone': 'Telefono', 'comp': 'Tecleo', 'silence': 'Silencio', 'dryer': 'Secadora', 
                'blender': 'Licuadora', 'doorbell': 'Timbre', 'alarm': 'Alarma', 'faucet': 'Grifo', 
                'microwave': 'Microondas'}
    features = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i], sr=8000)
        name = os.listdir(data_dir)[i]
        pattern = "[._][\w]+"
        name = re.sub(pattern, '', name)
        if name in momentos.keys():
            momento = momentos.get(name)
        else:
            momento = 'Otro'
        D = np.abs(np.fft.fft(y, n=10)[:8000*2 // 2])
        mfcc = np.ndarray.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20))
        print(sr)
        features.append([D[0], D[1], D[2], D[3], D[4], D[5], D[6], D[7], D[8], D[9], mfcc, momento])
    return pd.DataFrame(features, columns=['Fourier1', 'Fourier2', 'Fourier3', 'Fourier4', 'Fourier5', 
                                           'Fourier6', 'Fourier7', 'Fourier8', 'Fourier9', 'Fourier10', 
                                           'mfcc', 'momento'])

data = get_features('./sounds2/', glob('./sounds2' + '/*.wav'))

8000
8000
8000


In [11]:
def show_df_info(dataframe):
    display("The type of the dataframe is {}.".format(type(dataframe)))
    print("------------------------------------------------------------------------")
    display("The amount of entries are {}.".format(dataframe.size))
    print("------------------------------------------------------------------------")
    display("Its shape is {}.".format(dataframe.shape))
    print("------------------------------------------------------------------------")
    display("Its features are: {}.".format(list(dataframe)))
    print("------------------------------------------------------------------------")
    print("The data types of columns are: {}".format(dataframe.dtypes))
    print("------------------------------------------------------------------------")
    print("The counter of each class is: {}".format(dataframe.momento.value_counts()))
    print("------------------------------------------------------------------------")
    print('The amount of null values is: {}.'.format(dataframe.isna().sum()))
    display(dataframe)
    display(dataframe.describe())
    
show_df_info(data)

"The type of the dataframe is <class 'pandas.core.frame.DataFrame'>."

------------------------------------------------------------------------


'The amount of entries are 9.'

------------------------------------------------------------------------


'Its shape is (3, 3).'

------------------------------------------------------------------------


"Its features are: ['D', 'mfcc', 'momento']."

------------------------------------------------------------------------
The data types of columns are: D           object
mfcc       float64
momento     object
dtype: object
------------------------------------------------------------------------
The counter of each class is: Otro    3
Name: momento, dtype: int64
------------------------------------------------------------------------
The amount of null values is: D          0
mfcc       0
momento    0
dtype: int64.


,D,mfcc,momento
0,"[[0.002371622, 0.0010023934, 0.00020817871, 0....",-3.848597,Otro
1,"[[1.3690411, 1.6039556, 3.3537927, 2.32991, 4....",-0.390428,Otro
2,"[[50.140614, 29.09889, 7.7536716, 2.2107341, 3...",-0.235068,Otro


,mfcc
count,3.000000
mean,-1.491364
std,2.042901
min,-3.848597
25%,-2.119513
50%,-0.390428
75%,-0.312748
max,-0.235068


In [87]:
data['A'], data['B'], data['C'] = data['D'].str.split(',')

In [110]:
data

,Fourier1,Fourier2,Fourier3,Fourier4,Fourier5,Fourier6,Fourier7,Fourier8,Fourier9,Fourier10,mfcc,momento
0,0.000092,0.000031,0.000076,0.000031,0.000041,0.000031,0.000041,0.000031,0.000076,0.000031,-2.130798,Otro
1,0.798218,0.030360,0.056974,0.030780,0.013632,0.008911,0.013632,0.030780,0.056974,0.030360,1.838263,Otro
2,0.409149,0.126351,0.057948,0.040650,0.020767,0.018768,0.020767,0.040650,0.057948,0.126351,1.866691,Otro


In [104]:
data.Fourier6[0]

3.0517578125e-05

In [49]:
len(fourier1)/10

44177.5

In [34]:
fourier1 = []
for i in data.D[0]:
    for j in i:
        fourier1.append(j)

In [47]:
a = 0
b = 44177
lista1 = []
for i in range(10):
    y = sum(fourier1[a:b])
    lista1.append(y)
    a += 44177
    b += 44177
    

In [48]:
lista1

[103290.73543502635,
 13887.204518775387,
 1033.6304840533805,
 16.191152160835546,
 2.5612409457986622,
 1.6539086996433348,
 1.2690775203940878,
 1.0465804311976052,
 0.9425176184715518,
 0.8834538457045508]

In [35]:
fourier2 = []
for i in data.D[1]:
    for j in i:
        fourier2.append(j)

In [ ]:
a = 0
b = 44177
lista2 = []
for i in range(10):
    y = sum(fourier2[a:b])
    lista2.append(y)
    a += 44177
    b += 44177

In [ ]:
lista2

In [36]:
fourier3 = []
for i in data.D[2]:
    for j in i:
        fourier3.append(j)

In [37]:
len(fourier1)

441775

In [38]:
len(fourier2)

441775

In [39]:
len(fourier3)

441775

In [29]:
data['fourier'] = fourier

ValueError: Length of values does not match length of index

In [22]:
data.D[0]

array([[2.3716220e-03, 1.0023934e-03, 2.0817871e-04, ..., 1.1261877e+01,
        6.3997693e+00, 2.5181681e+01],
       [1.8741221e-03, 1.4557991e-03, 9.4269658e-04, ..., 6.6576471e+00,
        2.3446825e+00, 2.3329836e+01],
       [4.1683170e-04, 1.0886100e-03, 8.4229902e-04, ..., 1.5184585e+01,
        1.6317823e+01, 3.6874058e+01],
       ...,
       [2.9475952e-06, 1.4737884e-06, 5.4705979e-11, ..., 1.6898531e-07,
        7.0781138e-04, 7.7505321e-03],
       [2.9475484e-06, 1.4737598e-06, 5.9606924e-11, ..., 4.1855199e-07,
        7.0770690e-04, 7.7509005e-03],
       [2.9475368e-06, 1.4737822e-06, 4.7265702e-11, ..., 8.5414683e-08,
        7.0872583e-04, 7.7518215e-03]], dtype=float32)

In [20]:
len(fourier)

1025

In [9]:
len(data.loc[0, 'D'])

1025

In [12]:
len(data.loc[0, 'D'][0])

431

In [18]:
data.loc[0, 'D'][0]

array([2.37162202e-03, 1.00239343e-03, 2.08178710e-04, 1.48735265e-03,
       1.24947447e-03, 5.60650136e-04, 5.71757229e-03, 1.93187483e-02,
       2.69554388e-02, 2.39374749e-02, 1.08702015e-02, 3.65813188e-02,
       4.47993614e-02, 4.25000762e-04, 1.95175447e-02, 9.73172137e-04,
       1.53311947e-02, 2.81732772e-02, 2.27646381e-02, 1.08562768e-01,
       2.21774913e-02, 1.99333802e-02, 4.43793386e-02, 8.05871263e-02,
       1.48289025e-01, 1.76413879e-01, 7.17183053e-02, 1.58139318e-01,
       9.99311656e-02, 2.13208944e-01, 9.96383131e-02, 1.51793331e-01,
       3.81628156e-01, 2.04675406e-01, 1.25767767e-01, 1.08287983e-01,
       1.38229534e-01, 1.68189391e-01, 5.16825467e-02, 1.26205347e-02,
       2.42036972e-02, 4.22102101e-02, 2.00393930e-01, 9.80732590e-02,
       3.83197010e-01, 6.41518235e-01, 6.23247683e-01, 1.22885415e-02,
       4.36624527e-01, 6.62776753e-02, 7.67944306e-02, 6.96019411e-01,
       1.40179768e-01, 5.53346455e-01, 8.93449336e-02, 8.95094499e-02,
      

In [13]:
len(data.loc[1, 'D'])

1025

In [14]:
len(data.loc[1, 'D'][0])

431

In [15]:
len(data.loc[2, 'D'])

1025

In [16]:
len(data.loc[2, 'D'][0])

431

In [22]:
#data.to_csv('data_fourier.csv', index=False)